In [1]:
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# Import Fashion MNIST
fashion_mnist = input_data.read_data_sets('../../data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ../../data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ../../data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [3]:
"Training set (images) shape: {shape}".format(shape=fashion_mnist.train.images.shape)

'Training set (images) shape: (55000, 784)'

In [4]:
"Training set (labels) shape: {shape}".format(shape=fashion_mnist.train.labels.shape)

'Training set (labels) shape: (55000, 10)'

In [5]:
"Test set (images) shape: {shape}".format(shape=fashion_mnist.test.images.shape)

'Test set (images) shape: (10000, 784)'

In [6]:
"Test set (labels) shape: {shape}".format(shape=fashion_mnist.test.labels.shape)

'Test set (labels) shape: (10000, 10)'

In [7]:
seed = 777
tf.set_random_seed(seed)

In [8]:
def predictAccuracy(H, Y): # 예측치와 정확도 함수
    prediction = tf.equal(tf.argmax(H, axis=1), tf.argmax(Y, axis=1))
    accuracy   = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return prediction, accuracy

def getCost(mylogits, mylabel): # 비용 함수를 구해주는 함수
    diff = tf.nn.softmax_cross_entropy_with_logits_v2(logits= mylogits, labels = mylabel)
    cost = tf.reduce_mean(diff)
    return cost

def getOptimizer(learn_rate): # 옵티마이저 구하기
    optimizer = tf.train.AdamOptimizer(learning_rate = learn_rate)
    train     = optimizer.minimize(cost)
    return train

In [20]:
# PlaceHolder 설정

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.int32)

ph_keep  = tf.placeholder(tf.float32)

In [12]:
# 모델 구성

# 1번째 layer

# [3,3 → Filter의미, ,1 → 흑백 channel 한개,  32] → 32개의 Filter
w_conv_1 = tf.Variable(tf.truncated_normal([3, 3, 1, 32], stddev=0.1)) # channel의 1 → 32로 증가한다(두꺼워진다)
b_conv_1 = tf.Variable(tf.constant(0.1, shape=[32]))

l_conv_1 = tf.nn.conv2d(X, w_conv_1, strides=[1, 1, 1, 1], padding='SAME')
l_relu_1 = tf.nn.relu(l_conv_1 + b_conv_1)
l_pool_1 = tf.nn.max_pool(l_relu_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

Instructions for updating:
Colocations handled automatically by placer.


In [13]:
# 2번째 layer

w_conv_2 = tf.Variable(tf.truncated_normal([3, 3, 32, 64], stddev=0.1))
b_conv_2 = tf.Variable(tf.constant(0.1, shape=[64]))

l_conv_2 = tf.nn.conv2d(l_pool_1, w_conv_2, strides=[1, 1, 1, 1], padding='SAME')
l_relu_2 = tf.nn.relu(l_conv_2 + b_conv_2)
l_pool_2 = tf.nn.max_pool(l_relu_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [16]:
# 3번째 flat Layer

l_flat   = tf.reshape(l_pool_2, [-1, 7 * 7 * 64])

w_full_3 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 256], stddev=0.1))
b_full_3 = tf.Variable(tf.constant(0.1, shape=[256]))

l_full_3 = tf.matmul(l_flat, w_full_3) + b_full_3
l_relu_3 = tf.nn.relu(l_full_3)
l_drop_3 = tf.nn.dropout(l_relu_3, rate=0.7)

In [18]:
# last

w_full_4 = tf.Variable(tf.truncated_normal([256, 10], stddev=0.1))
b_full_4 = tf.Variable(tf.constant(0.1, shape=[10]))

# (?, 10)
logits = tf.matmul(l_drop_3, w_full_4) + b_full_4
H      = tf.nn.softmax(logits)

In [19]:
# Hyper Parameter Setting

learn_rate = 0.001
epochs     = 1
batch_size = 100
batch_iter = fashion_mnist.train.num_examples // batch_size # 550

In [21]:
cost = getCost(logits, Y)

train = getOptimizer( learn_rate )

In [22]:
# Train

sess = tf.Session()
sess.run(tf.global_variables_initializer())

# epcochs를 1번 돌고 념
for i in range(epochs):
    
    total_cost = 0 #비용 전체의 합
    
    for j in range(batch_iter):
        batch_xs, batch_ys = fashion_mnist.train.next_batch(batch_size)

        batch_xs = batch_xs.reshape([-1, 28, 28, 1])

        feed_data = {X: batch_xs, Y: batch_ys, ph_keep: 0.5}
        _cost, _train = sess.run([cost, train], feed_dict=feed_data)
        
        total_cost += _cost        

    # avg_cost : 평균비용
    avg_cost = total_cost / batch_iter        
#     print('{:.9f}'.format(avg_cost))
    print('Epoch {0} :   {1:.9f}'.format(i, avg_cost))    

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float
	 [[node Placeholder (defined at <ipython-input-11-91ba5e09504e>:3) ]]

Caused by op 'Placeholder', defined at:
  File "C:\Users\shpim\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\shpim\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\shpim\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\shpim\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\shpim\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\Users\shpim\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\shpim\Anaconda3\lib\asyncio\base_events.py", line 539, i

In [ ]:
xx = fashion_mnist.test.images.reshape([-1, 28, 28, 1])

preds = sess.run(H, {X: xx, ph_keep: 1.0})

In [ ]:
preds_arg = np.argmax(preds, axis=1)  # 1: 수평, 0: 수직
# spase일때는 argmax를 가져올필요 없다.
test_arg  = np.argmax(fashion_mnist.test.labels, axis=1)

print('acc: ', np.mean(preds_arg == test_arg))

# # 모델 테스트와 정확도 체크하기
# prediction, accuracy = predictAccuracy(H, Y)

# print('\n 정확도:', end='')

# feed_data = {X:fashion_mnist.mnist.test.images, Y:fashion_mnist.mnist.test.labels, dropout_prob:1.0}

# print(sess.run([accuracy], feed_dict= feed_data))

In [ ]:
# 임의의 그림 1개를 구하고, 예측하기
randitem  = random.randint(0, fashion_mnist.test.num_examples -1)
print('\n라벨(Label) :', end=' ')
print(sess.run(tf.argmax(fashion_mnist.test.labels[randitem:randitem+1], axis=1)))

print('\n예측(Prediction) :', end=' ')
feed_data={X:mnist.test.images[randitem:randitem+1], dropout_prob:1.0}
print(sess.run(tf.argmax(H, axis=1), feed_dict = feed_data))

# plt.imshow(fashion_mnist.test.images[randitem:randitem+1].reshape(img_row, img_column), cmap='Greys', interpolation='nearest')
# plt.show()

In [ ]:
sess.close()